In [343]:
from collections import defaultdict
from datetime import datetime
import random

import numpy as np
import pandas as pd

import spacy
nlp = spacy.load('en_core_web_sm')

from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES
lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)

import neuralcoref
nlp.add_pipe(neuralcoref.NeuralCoref(nlp.vocab,blacklist=False),name="neuralcoref")

from main2 import ConnoFramer

In [344]:
framer = ConnoFramer()
framer.train('/Users/maria/Documents/data/FramesAgencyPower/agency_power.csv', 
             'fakeStories.csv', 
             'text', 
             'i')

2023-02-04 21:18:14 Processed 0 out of 2


In [345]:
framer.get_score_totals()

defaultdict(<function main2.ConnoFramer.__score_dataset.<locals>.<lambda>()>,
            {'i': defaultdict(int, {'positive': 1, 'negative': 2}),
             'my doctor': defaultdict(int, {'positive': 4, 'negative': 0}),
             'susan': defaultdict(int, {'positive': 0, 'negative': 1})})

In [346]:
framer.get_scores_for_doc(1)

defaultdict(<function main2.ConnoFramer.__score_document.<locals>.<lambda>()>,
            {'susan': defaultdict(int, {'negative': 1, 'positive': 0}),
             'my doctor': defaultdict(int, {'positive': 3, 'negative': 0})})

In [347]:
framer.count_nsubj_for_doc(1)

defaultdict(int,
            {('susan', 'arrive'): 1,
             ('susan', 'call'): 1,
             ('my doctor', 'fix'): 1})

In [348]:
framer.count_dobj_for_doc(1)

defaultdict(int, {('my doctor', 'thank'): 1, ('my doctor', 'call'): 1})

In [349]:
framer.get_scores_for_doc(0)

defaultdict(<function main2.ConnoFramer.__score_document.<locals>.<lambda>()>,
            {'i': defaultdict(int, {'positive': 1, 'negative': 2}),
             'my doctor': defaultdict(int, {'positive': 1, 'negative': 0})})

In [350]:
framer.count_nsubj_for_doc(0)

defaultdict(int,
            {('i', 'think'): 1,
             ('i', 'have'): 1,
             ('i', 'feel'): 1,
             ('i', 'call'): 1})

In [351]:
framer.count_dobj_for_doc(0)

defaultdict(int, {('i', 'pick'): 1, ('my doctor', 'call'): 1})